In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
from splinter import Browser
import pandas as pd
import itertools
from datetime import datetime

In [2]:
team_csv = pd.read_csv('team_name_lookup.csv')
team_csv

,team_id,team_abbrev,team_full,league,division
0,1,ARI,Arizona Diamondbacks,NL,WEST
1,2,ATL,Atlanta Braves,NL,EAST
2,3,BAL,Baltimore Orioles,AL,EAST
3,4,BOS,Boston Red Sox,AL,EAST
4,5,CHW,Chicago White Sox,AL,CENTRAL
5,6,CHC,Chicago Cubs,NL,CENTRAL
6,7,CIN,Cincinnati Reds,NL,CENTRAL
7,8,CLE,Cleveland Indians,AL,CENTRAL
8,9,COL,Colorado Rockies,NL,WEST
9,10,DET,Detroit Tigers,AL,CENTRAL


In [3]:
player_csv = pd.read_csv('Washington_Players')
player_csv

,playerID,playerName,teamID,majorsFlag,fortyManFlag,injuryStatus,positions,bats,throws,height,weight,dateOfBirth,serviceTime,imgUrl
0,baldalb01,Alberto Baldonado,30.0,1.0,1.0,NaN,Pitcher,L,L,76,250,"February 1, 1993",0.000,https://www.baseball-reference.com/req/2021080...
1,finnkyl01,Kyle Finnegan,30.0,1.0,1.0,NaN,Pitcher,R,R,74,200,"September 4, 1991",1.000,https://www.baseball-reference.com/req/2021080...
2,grayjos01,Josiah Gray,30.0,1.0,1.0,NaN,Pitcher,R,R,73,190,"December 21, 1997",0.000,https://www.baseball-reference.com/req/2021080...
3,thommas01,Mason Thompson,30.0,1.0,1.0,NaN,Pitcher,R,R,79,223,"February 20, 1998",0.000,https://www.baseball-reference.com/req/2021080...
4,espipao01,Paolo Espino,30.0,1.0,1.0,NaN,Pitcher,R,R,70,215,"January 10, 1987",0.078,https://www.baseball-reference.com/req/2021080...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,sancadr01,Adrian Sanchez,30.0,0.0,0.0,NaN,"Pinch Hitter, Second Baseman and Third Baseman",R,R,72,208,"August 16, 1990",1.083,https://www.baseball-reference.com/req/2021080...
61,schemax01,Max Scherzer,14.0,1.0,1.0,NaN,Pitcher,R,R,75,208,"July 27, 1984",12.079,https://www.baseball-reference.com/req/2021080...
62,schwkyl01,Kyle Schwarber,4.0,1.0,1.0,NaN,Leftfielder and Catcher,L,R,72,229,"March 5, 1993",5.086,https://www.baseball-reference.com/req/2021080...
63,turntre01,Trea Turner,14.0,1.0,1.0,NaN,"Shortstop, Second Baseman and Centerfielder",R,R,74,185,"June 30, 1993",4.135,https://www.baseball-reference.com/req/2021080...


In [4]:
#Define Executable Path for ChromeDriver to use for web scraping

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
# Define base endpoint URL. Other url's will be built off this endpoint.

baseball_reference_main = 'https://www.baseball-reference.com'
nats_schedule_endpoint = '/teams/WSN/2021-schedule-scores.shtml'
nats_schedule_page = baseball_reference_main + nats_schedule_endpoint
browser.visit(nats_schedule_page)

In [6]:
#Use Beautiful Soup HTML.parser to extract relvant details about news articles

schedule_html = browser.html
schedule_soup = bs(schedule_html, 'html.parser')
schedule_soup

<html class="js cookies localstorage sessionstorage cors history csspositionsticky no-touchevents pointerevents matchmedia flexwrap desktop is_live" data-root="/home/br/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en" style=""><head>
<script src="https://partner.googleadservices.com/gampad/cookie.js?domain=www.baseball-reference.com&amp;callback=_gfp_s_&amp;client=ca-pub-5319453360923253"></script><script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script src="//js-sec.indexww.com/ht/p/184310-82987131453484.js"></script><script src="https://freestar-io.videoplayerhub.com/gallery.js"></script><script src="https://pghub.io/js/pandg-sdk.js"></script><meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202112021" rel="dns-prefetch"

In [7]:
# Extract list of games

game_listings = schedule_soup.find('table', class_="sortable stats_table now_sortable", id="team_schedule").find('tbody').find_all('tr',class_=lambda x: x != 'thead')
game_listings

[<tr data-row="0"><th class="right" data-stat="team_game" scope="row">1</th><td class="left" csk="2021-04-06" data-stat="date_game"><a href="/boxes/?date=2021-04-06" name="1" title="Click for standings on this date">Tuesday, Apr 6</a></td><td class="left" data-stat="boxscore"><a href="/boxes/WAS/WAS202104060.shtml">boxscore</a></td><td class="left" data-stat="team_ID">WSN</td><td class="left iz" data-stat="homeORvis"></td><td class="left" data-stat="opp_ID"><a href="/teams/ATL/2021.shtml">ATL</a></td><td class="left" data-stat="win_loss_result">W-wo</td><td class="right" data-stat="R">6</td><td class="right" data-stat="RA">5</td><td class="right iz" csk="8.50" data-stat="extra_innings"></td><td class="right" csk="1.0" data-stat="win_loss_record">1-0</td><td class="right" data-stat="rank">2</td><td class="right" csk="2" data-stat="games_back"> 1.0</td><td class="left" data-stat="winning_pitcher"><a href="/players/h/hudsoda01.shtml" title="Daniel Hudson">Hudson</a></td><td class="left" d

In [8]:
game_listings[30]

<tr data-row="31"><th class="right" data-stat="team_game" scope="row">31</th><td class="left" csk="2021-05-11" data-stat="date_game"><a href="/boxes/?date=2021-05-11" name="31" title="Click for standings on this date">Tuesday, May 11</a></td><td class="left" data-stat="boxscore"><a href="/boxes/WAS/WAS202105110.shtml">boxscore</a></td><td class="left" data-stat="team_ID">WSN</td><td class="left iz" data-stat="homeORvis"></td><td class="left" data-stat="opp_ID"><a href="/teams/PHI/2021.shtml">PHI</a></td><td class="left" data-stat="win_loss_result">L</td><td class="right" data-stat="R">2</td><td class="right" data-stat="RA">6</td><td class="right iz" csk="9.00" data-stat="extra_innings"></td><td class="right" csk="-5.0" data-stat="win_loss_record">13-18</td><td class="right" data-stat="rank">4</td><td class="right" csk="10" data-stat="games_back"> 5.0</td><td class="left" data-stat="winning_pitcher"><a href="/players/a/anderch01.shtml" title="Chase Anderson">Anderson</a></td><td class="

In [9]:
GAME_info = []
GAME_beg_columns = ['gameDateFull','gameYear','awayTeam','awayScore','homeTeam','homeScore']
HIT_info = []
HIT_beg_columns = []
PITCH_beg_columns = []
PITCH_info = []
game_number = 0
limit = 1

for index, game in enumerate(game_listings[30:]):
    if index == limit:
        break
    game_number += 1
    boxscore = game.find_all('td')[1]
    boxscore_validation = boxscore.text
    if boxscore_validation == 'boxscore':
        boxscore_endpoint = boxscore.a['href']
        boxscore_url = baseball_reference_main + boxscore_endpoint
        browser.visit(boxscore_url)
        box_html = browser.html
        box_soup = bs(box_html, 'html.parser')

        # General game details for GAME instances. Prior to additional cleaning

#         score_metadata = box_soup.find('div', class_ = 'scorebox_meta').find_all('div')
#         game_meta_data = []
#         for datapoint in score_metadata[:5]:
#             meta_contents = datapoint.contents
#             if len(meta_contents) > 1:
#                 key_value_split = meta_contents[1].split(": ")[-1]
#                 game_meta_data.append(key_value_split)
#             else:
#                 key_value_split = meta_contents[0].split(": ",1)[-1]
#                 game_meta_data.append(key_value_split)   
#         full_date = game_meta_data[0]
#         year = full_date.split(',')[-1].strip()
#         date_object = datetime.strptime(full_date, "%A, %B %d, %Y")
#         date_iso = date_object.date().isoformat()
#         gen_game_info = box_soup.find('table', class_ = 'linescore nohover stats_table no_freeze').find('tbody').find_all('tr')
#         gen_game_by_team = []
#         gen_game_by_team.append(date_iso)
#         gen_game_by_team.append(year)
#         for index, team in enumerate(gen_game_info):
#             data = team.find_all('td')
#             team_name = data[1].text.strip()
#             team_runs = data[-3].text.strip()
#             if index == 0:
#                 team_designation = 'AWAY'
#             else:
#                 team_designation = 'HOME'
#             gen_game_by_team.append(team_name)
#             gen_game_by_team.append(team_runs)
#         GAME_info.append(gen_game_by_team)

#         # Boxscore Hitting Statistics. Player-by-player

        nats_batting = box_soup.find('table', id="WashingtonNationalsbatting")
        
        if game_number == 1:
            nats_hitting_headers = box_soup.find('div', id="div_WashingtonNationalsbatting").find('thead').find('tr').find_all('th')
            HIT_beg_columns.append('GAME_ID')
            HIT_beg_columns.append('Name')
            HIT_beg_columns.append('Position')
            for header in nats_hitting_headers:
                one_header = header.text
                if one_header != 'Batting':
                    HIT_beg_columns.append(one_header) 
                else:
                    pass
        else:
            pass

        nats_hitstats_location = box_soup.find('div', id="div_WashingtonNationalsbatting").find('tbody')
        names_positions_stats = nats_hitstats_location.find_all('tr')

        for players in names_positions_stats:
            hit_list_atts = []
            hit_list_atts.append(game_number)
            name_and_position = players.find('th')
            player_content_length = len(name_and_position.contents)
            try:
                if player_content_length == 2:
                    player_name = name_and_position.contents[0].text
                    player_position = name_and_position.contents[1]
                else:
                    player_name = name_and_position.contents[1].text
                    player_position = name_and_position.contents[2]
                stats_for_players = players.find_all('td')
                hit_list_atts.append(player_name)
                hit_list_atts.append(player_position)
                for stat in stats_for_players:
                    each_stat = stat.text
                    hit_list_atts.append(each_stat)
            except IndexError:
                pass
            HIT_info.append(hit_list_atts)
            
        # Boxscore Pitching Statistics. Player-by-Player
        
        #nats_pitch_stats_location = box_soup.find('table', id="WashingtonNationalspitching")
        
#         if game_number == 1:
#             nats_pitch_headers = box_soup.find('div', id="div_WashingtonNationalspitching").find('thead').find('tr').find_all('th')
#             PITCH_beg_columns.append('GAME_ID')
#             for header in nats_pitch_headers:
#                 one_header = header.text
#                 PITCH_beg_columns.append(one_header)
#             PITCH_beg_columns.append('SPFlag')
#             PITCH_beg_columns.append('HBP')
#             PITCH_beg_columns.append('IBB')
#             PITCH_beg_columns.append('W')
#             PITCH_beg_columns.append('L')
#             PITCH_beg_columns.append('H.1')
#             PITCH_beg_columns.append('S')
#             PITCH_beg_columns.append('BS')
#         else:
#             pass
        
#         nats_pitchstats_location = box_soup.find('div', id="div_WashingtonNationalspitching").find('tbody')
#         names_pitch_stats = nats_pitchstats_location.find_all('tr')
#         hbp_indiv_events_html = box_soup.find('div', class_="indiv_events").find(string=re.compile("HBP:")).find_parent('div')
#         hbp_results = hbp_indiv_events_html.contents[1:]
#         hbp_string = hbp_results[0].replace(u'\xa0', u' ')
#         ibb_indiv_events_html = box_soup.find('div', class_="indiv_events").find(string=re.compile("IBB:")).find_parent('div')
#         ibb_results = ibb_indiv_events_html.contents[1:]
#         ibb_string = ibb_results[0].replace(u'\xa0', u' ')
#         All_pitchers_Info = []
#         for index_player, players in enumerate(names_pitch_stats):
#             pitch_list_atts = []
#             pitch_list_atts.append(game_number)
#             names_and_records = players.find('th')
#             player_content_length = len(names_and_records.contents)  
#             player_name = names_and_records.contents[0].text
#             pitch_list_atts.append(player_name)
#             stats_for_players = players.find_all('td')
#             for stat in stats_for_players:
#                 each_stat = stat.text
#                 pitch_list_atts.append(each_stat)
#             if index_player == 0:
#                 pitch_list_atts.append(1)
#             else:
#                 pitch_list_atts.append(0)
#             if player_name in hbp_string:
#                 player_beginning_loc = hbp_string.find(player_name)
#                 player_beginning = hbp_string[player_beginning_loc:]
#                 start_grouping = player_beginning.find("(") + len("(")
#                 end_grouping = player_beginning.find(")")
#                 if end_grouping > start_grouping & start_grouping != 0:
#                     hbp_log_instances = player_beginning[start_grouping:end_grouping]
#                     hbp_log_all_instances = hbp_log_instances.split(",")
#                     total_hbp = len(hbp_log_all_instances)
#                     pitch_list_atts.append(total_hbp)
#                 else:
#                     pitch_list_atts.append(0)
#                 #print(f'{player_name} hit {total_hbp} batter(s)')
#             else:
#                 #print(f'{player_name} did NOT hit anyone')
#                 pitch_list_atts.append(0)
#             if player_name in ibb_string:
#                 player_beginning_loc = ibb_string.find(player_name)
#                 player_beginning = ibb_string[player_beginning_loc:]
#                 start_grouping = player_beginning.find("(") + len("(")
#                 end_grouping = player_beginning.find(")")
#                 if end_grouping > start_grouping & start_grouping != 0:
#                     ibb_log_instances = player_beginning[start_grouping:end_grouping]
#                     ibb_log_all_instances = ibb_log_instances.split(",")
#                     total_ibb = len(ibb_log_all_instances)
#                     pitch_list_atts.append(total_ibb)
#                 else:
#                     pitch_list_atts.append(0)
#                 #print(f'{player_name} hit {total_hbp} batter(s)')
#             else:
#                 #print(f'{player_name} did NOT hit anyone')
#                 pitch_list_atts.append(0)
#             if player_content_length == 2:
#                 player_decision = names_and_records.contents[1]
#                 decision_split = player_decision.split()
#                 decisionsToCheck = ['W', 'L', 'H', 'S', 'BS']
#                 for decision in decisionsToCheck:
#                     if decision in decision_split:
#                         pitch_list_atts.append(1)
#                     else:
#                         pitch_list_atts.append(0)
#             else:
#                 pitch_list_atts.extend([0] * 5)
#             PITCH_info.append(pitch_list_atts)
        # Game Counter
        print(f'Game {game_number} scraped')
    else:
        print('Game has not yet occured')

Game 1 scraped


In [15]:
# GAME_info = []
# GAME_beg_columns = ['gameDateFull','gameYear','awayTeam','awayScore','homeTeam','homeScore']
# HIT_info = []
# HIT_beg_columns = []
# PITCH_beg_columns = []
# PITCH_info = []
limit = 200

for index, game in enumerate(game_listings[game_number:]):
    if index == limit:
        break
    game_number += 1
    boxscore = game.find_all('td')[1]
    boxscore_validation = boxscore.text
    if boxscore_validation == 'boxscore':
        boxscore_endpoint = boxscore.a['href']
        boxscore_url = baseball_reference_main + boxscore_endpoint
        browser.visit(boxscore_url)
        box_html = browser.html
        box_soup = bs(box_html, 'html.parser')

        # General game details for GAME instances. Prior to additional cleaning

#         score_metadata = box_soup.find('div', class_ = 'scorebox_meta').find_all('div')
#         game_meta_data = []
#         for datapoint in score_metadata[:5]:
#             meta_contents = datapoint.contents
#             if len(meta_contents) > 1:
#                 key_value_split = meta_contents[1].split(": ")[-1]
#                 game_meta_data.append(key_value_split)
#             else:
#                 key_value_split = meta_contents[0].split(": ",1)[-1]
#                 game_meta_data.append(key_value_split)   
#         full_date = game_meta_data[0]
#         year = full_date.split(',')[-1].strip()
#         date_object = datetime.strptime(full_date, "%A, %B %d, %Y")
#         date_iso = date_object.date().isoformat()
#         gen_game_info = box_soup.find('table', class_ = 'linescore nohover stats_table no_freeze').find('tbody').find_all('tr')
#         gen_game_by_team = []
#         gen_game_by_team.append(date_iso)
#         gen_game_by_team.append(year)
#         for index, team in enumerate(gen_game_info):
#             data = team.find_all('td')
#             team_name = data[1].text.strip()
#             team_runs = data[-3].text.strip()
#             if index == 0:
#                 team_designation = 'AWAY'
#             else:
#                 team_designation = 'HOME'
#             gen_game_by_team.append(team_name)
#             gen_game_by_team.append(team_runs)
#         GAME_info.append(gen_game_by_team)

#         # Boxscore Hitting Statistics. Player-by-player

        nats_batting = box_soup.find('table', id="WashingtonNationalsbatting")
        
        if game_number == 1:
            nats_hitting_headers = box_soup.find('div', id="div_WashingtonNationalsbatting").find('thead').find('tr').find_all('th')
            HIT_beg_columns.append('GAME_ID')
            HIT_beg_columns.append('Name')
            HIT_beg_columns.append('Position')
            for header in nats_hitting_headers:
                one_header = header.text
                if one_header != 'Batting':
                    HIT_beg_columns.append(one_header) 
                else:
                    pass
        else:
            pass

        nats_hitstats_location = box_soup.find('div', id="div_WashingtonNationalsbatting").find('tbody')
        names_positions_stats = nats_hitstats_location.find_all('tr')

        for players in names_positions_stats:
            hit_list_atts = []
            hit_list_atts.append(game_number)
            name_and_position = players.find('th')
            player_content_length = len(name_and_position.contents)
            try:
                if player_content_length == 2:
                    player_name = name_and_position.contents[0].text
                    player_position = name_and_position.contents[1]
                else:
                    player_name = name_and_position.contents[1].text
                    player_position = name_and_position.contents[2]
                stats_for_players = players.find_all('td')
                hit_list_atts.append(player_name)
                hit_list_atts.append(player_position)
                for stat in stats_for_players:
                    each_stat = stat.text
                    hit_list_atts.append(each_stat)
            except IndexError:
                pass
            HIT_info.append(hit_list_atts)
            
        # Boxscore Pitching Statistics. Player-by-Player
        
        #nats_pitch_stats_location = box_soup.find('table', id="WashingtonNationalspitching")
        
#         if game_number == 1:
#             nats_pitch_headers = box_soup.find('div', id="div_WashingtonNationalspitching").find('thead').find('tr').find_all('th')
#             PITCH_beg_columns.append('GAME_ID')
#             for header in nats_pitch_headers:
#                 one_header = header.text
#                 PITCH_beg_columns.append(one_header)
#             PITCH_beg_columns.append('SPFlag')
#             PITCH_beg_columns.append('HBP')
#             PITCH_beg_columns.append('IBB')
#             PITCH_beg_columns.append('W')
#             PITCH_beg_columns.append('L')
#             PITCH_beg_columns.append('H.1')
#             PITCH_beg_columns.append('S')
#             PITCH_beg_columns.append('BS')
#         else:
#             pass
        
#         nats_pitchstats_location = box_soup.find('div', id="div_WashingtonNationalspitching").find('tbody')
#         names_pitch_stats = nats_pitchstats_location.find_all('tr')
#         hbp_indiv_events_html = box_soup.find('div', class_="indiv_events").find(string=re.compile("HBP:")).find_parent('div')
#         hbp_results = hbp_indiv_events_html.contents[1:]
#         hbp_string = hbp_results[0].replace(u'\xa0', u' ')
#         ibb_indiv_events_html = box_soup.find('div', class_="indiv_events").find(string=re.compile("IBB:")).find_parent('div')
#         ibb_results = ibb_indiv_events_html.contents[1:]
#         ibb_string = ibb_results[0].replace(u'\xa0', u' ')
#         All_pitchers_Info = []
#         for index_player, players in enumerate(names_pitch_stats):
#             pitch_list_atts = []
#             pitch_list_atts.append(game_number)
#             names_and_records = players.find('th')
#             player_content_length = len(names_and_records.contents)  
#             player_name = names_and_records.contents[0].text
#             pitch_list_atts.append(player_name)
#             stats_for_players = players.find_all('td')
#             for stat in stats_for_players:
#                 each_stat = stat.text
#                 pitch_list_atts.append(each_stat)
#             if index_player == 0:
#                 pitch_list_atts.append(1)
#             else:
#                 pitch_list_atts.append(0)
#             if player_name in hbp_string:
#                 player_beginning_loc = hbp_string.find(player_name)
#                 player_beginning = hbp_string[player_beginning_loc:]
#                 start_grouping = player_beginning.find("(") + len("(")
#                 end_grouping = player_beginning.find(")")
#                 if end_grouping > start_grouping & start_grouping != 0:
#                     hbp_log_instances = player_beginning[start_grouping:end_grouping]
#                     hbp_log_all_instances = hbp_log_instances.split(",")
#                     total_hbp = len(hbp_log_all_instances)
#                     pitch_list_atts.append(total_hbp)
#                 else:
#                     pitch_list_atts.append(0)
#                 #print(f'{player_name} hit {total_hbp} batter(s)')
#             else:
#                 #print(f'{player_name} did NOT hit anyone')
#                 pitch_list_atts.append(0)
#             if player_name in ibb_string:
#                 player_beginning_loc = ibb_string.find(player_name)
#                 player_beginning = ibb_string[player_beginning_loc:]
#                 start_grouping = player_beginning.find("(") + len("(")
#                 end_grouping = player_beginning.find(")")
#                 if end_grouping > start_grouping & start_grouping != 0:
#                     ibb_log_instances = player_beginning[start_grouping:end_grouping]
#                     ibb_log_all_instances = ibb_log_instances.split(",")
#                     total_ibb = len(ibb_log_all_instances)
#                     pitch_list_atts.append(total_ibb)
#                 else:
#                     pitch_list_atts.append(0)
#                 #print(f'{player_name} hit {total_hbp} batter(s)')
#             else:
#                 #print(f'{player_name} did NOT hit anyone')
#                 pitch_list_atts.append(0)
#             if player_content_length == 2:
#                 player_decision = names_and_records.contents[1]
#                 decision_split = player_decision.split()
#                 decisionsToCheck = ['W', 'L', 'H', 'S', 'BS']
#                 for decision in decisionsToCheck:
#                     if decision in decision_split:
#                         pitch_list_atts.append(1)
#                     else:
#                         pitch_list_atts.append(0)
#             else:
#                 pitch_list_atts.extend([0] * 5)
#             PITCH_info.append(pitch_list_atts)
        # Game Counter
        print(f'Game {game_number} scraped')
    else:
        print('Game has not yet occured')

Game 121 scraped
Game 122 scraped
Game 123 scraped
Game 124 scraped
Game 125 scraped
Game 126 scraped
Game 127 scraped
Game 128 scraped
Game 129 scraped
Game 130 scraped
Game 131 scraped
Game 132 scraped
Game 133 scraped
Game 134 scraped
Game 135 scraped
Game 136 scraped
Game 137 scraped
Game 138 scraped
Game 139 scraped
Game 140 scraped
Game 141 scraped
Game 142 scraped
Game 143 scraped
Game 144 scraped
Game 145 scraped
Game 146 scraped
Game 147 scraped
Game 148 scraped
Game 149 scraped
Game 150 scraped
Game 151 scraped
Game 152 scraped
Game 153 scraped
Game 154 scraped
Game 155 scraped
Game 156 scraped
Game 157 scraped
Game 158 scraped
Game 159 scraped
Game 160 scraped
Game 161 scraped
Game 162 scraped


In [11]:
# HIT_info

In [ ]:
# GAME_df = pd.DataFrame(GAME_info, columns = GAME_beg_columns)
# GAME_df

In [ ]:
# away_team_list = GAME_df['awayTeam'].to_list()
# home_team_list = GAME_df['homeTeam'].to_list()

# awayTeamId = []
# homeTeamId = []

# for team in away_team_list:
#     team_unique_id = team_csv.loc[team_csv['team_full'] == team, 'team_id'].iloc[0]
#     awayTeamId.append(team_unique_id)

# for team in home_team_list:
#     team_unique_id = team_csv.loc[team_csv['team_full'] == team,'team_id'].iloc[0]
#     homeTeamId.append(team_unique_id)
    

# GAME_df['awayTeam'] = awayTeamId
# GAME_df['homeTeam'] = homeTeamId

In [ ]:
# GAME_df

In [ ]:
# gameUniqueId = []
# for index, row in GAME_df.iterrows():
#     away_team = row['awayTeam']
#     home_team = row['homeTeam']
#     date_value = row['gameDateFull']
#     if away_team >= 10:
#         away_team_twodig = str(away_team)
#     else:
#         away_team_twodig = '0' + str(away_team)
#     if home_team >= 10:
#         home_team_twodig = str(home_team)
#     else:
#         home_team_twodig = '0' + str(home_team)
    
#     new_date = date_value.replace("-","")

#     primary_id = new_date+away_team_twodig+home_team_twodig+'01'
    
#     if primary_id in gameUniqueId:
#         secondary_id = new_date+away_team_twodig+home_team_twodig+'02'
#         gameUniqueId.append(secondary_id)
#     else:
#         gameUniqueId.append(primary_id)

In [ ]:
# GAME_df.insert(0, 'gameId', gameUniqueId)

In [ ]:
# GAME_df

In [ ]:
# GAME_df.to_csv('GAME', index=False)  

In [10]:
GAME_df = pd.read_csv('GAME')
GAME_df

,gameId,gameDateFull,gameYear,awayTeam,awayScore,homeTeam,homeScore
0,20210406023001,2021-04-06,2021,2,5,30,6
1,20210407023001,2021-04-07,2021,2,7,30,6
2,20210407023002,2021-04-07,2021,2,2,30,0
3,20210409301401,2021-04-09,2021,30,0,14,1
4,20210410301401,2021-04-10,2021,30,5,14,9
...,...,...,...,...,...,...,...
157,20210928300901,2021-09-28,2021,30,1,9,3
158,20210929300901,2021-09-29,2021,30,5,9,10
159,20211001043001,2021-10-01,2021,4,4,30,2
160,20211002043001,2021-10-02,2021,4,5,30,3


In [11]:
HIT_df = pd.DataFrame(HIT_info, columns = HIT_beg_columns)

In [12]:
HIT_df

,GAME_ID,Name,Position,AB,R,H,RBI,BB,SO,PA,...,WPA,aLI,WPA+,WPA-,cWPA,acLI,RE24,PO,A,Details
0,1,Trea Turner,SS,4,1,3,1,1,1,5,...,0.141,0.81,0.158,-0.018%,0.07%,0.73,1.6,2,0,HR
1,1,Juan Soto,RF,2,1,1,0,3,0,5,...,0.161,1.52,0.173,-0.012%,0.09%,1.36,1.3,1,0,
2,1,Josh Bell,1B,4,0,0,0,0,2,4,...,-0.268,2.89,0.000,-0.268%,-0.14%,2.59,-1.8,4,1,
3,1,Kyle Finnegan,P,0,0,0,0,0,0,0,...,,,,,,,,0,1,
4,1,Paolo Espino,P,0,0,0,0,0,0,0,...,,,,,,,,0,0,
5,1,Andrew Stevenson,PH,1,0,0,0,0,1,1,...,-0.014,0.51,0.000,-0.014%,-0.01%,0.46,-0.5,,,
6,1,Kyle Schwarber,LF,3,0,1,1,0,0,4,...,0.082,1.30,0.144,-0.062%,0.04%,1.17,0.4,5,0,HBP
7,1,Yan Gomes,C,4,0,0,0,0,1,4,...,-0.133,1.37,0.000,-0.133%,-0.07%,1.23,-1.2,7,1,
8,1,Starlin Castro,3B,4,0,1,0,0,0,4,...,-0.083,0.86,0.007,-0.090%,-0.04%,0.77,-0.6,2,3,
9,1,Josh Harrison,2B,4,0,0,0,0,2,4,...,-0.102,0.99,0.000,-0.102%,-0.06%,0.90,-0.8,1,0,


In [45]:
HIT_df.loc[HIT_df['AB'].isnull()]

,GAME_ID,Name,Position,AB,R,H,RBI,BB,SO,PA,...,WPA,aLI,WPA+,WPA-,cWPA,acLI,RE24,PO,A,Details
287,20,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
302,21,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
401,28,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
418,29,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
433,30,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
447,31,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
827,57,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
844,58,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1390,96,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1405,97,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [46]:
HIT_df = HIT_df.dropna()

In [47]:
HIT_df.loc[HIT_df['AB'].isnull()]

,GAME_ID,Name,Position,AB,R,H,RBI,BB,SO,PA,...,WPA,aLI,WPA+,WPA-,cWPA,acLI,RE24,PO,A,Details


In [48]:
HIT_df['GAME_ID'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162])

In [49]:
HIT_details_filter = HIT_df['Details'].to_list()

ultimate_hitOutcome_list = []
for index, record in enumerate(HIT_details_filter):
    hOutcomesToCheck = ['HR', '2B', '3B', 'HBP', 'IW','SF']
    hOutcomesPerPlayer = []
    if index >= 0:
        try:
            hOutcomesPerPlayer.extend([0] * len(hOutcomesToCheck))
            outcome_values = record.split(',')
            print(outcome_values)
            for outcome in hOutcomesToCheck:
                #Data quality issue is below. Not capturing it. Need to figure out
                for event in outcome_values:
                    if outcome in event:
                        index_to_insert = hOutcomesToCheck.index(outcome)
                        num_outcome_per_value = re.split('[· ]', event)[0]
                        if num_outcome_per_value == event:
                            num_to_insert  = 1
                            hOutcomesPerPlayer[index_to_insert] = num_to_insert
                        else:
                            num_to_insert = int(num_outcome_per_value)
                            hOutcomesPerPlayer[index_to_insert] = num_to_insert
                    else:
                        pass
        except NameError:
            pass

        hOutcomes_tupelized = tuple(hOutcomesPerPlayer)
        ultimate_hitOutcome_list.append(hOutcomes_tupelized)
        
    else:
        pass
            
print(ultimate_hitOutcome_list)

# items_tfo = ['1B','2B','3B','HR']
# list_2 = ['9 | 2B','HBP']
# for item in items_tfo:
#     for event in list_2:
#         if item in event:
#             print(f'{item} found')
#         else:
#             print(f'{item} not found')

['']
['HR', 'HBP']
['']
['']
['']
['']
['']
['']
['2B']
['']
['']
['']
['']
['']
['']
['']
['HBP']
['HR', '2B']
['']
['2B']
['2B']
['']
['']
['SF', 'GDP']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['SB']
['GDP']
['SB', 'IW']
['']
['']
['']
['']
['']
['']
['2B']
['']
['']
['CS']
['']
['CS', 'IW']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['2B']
['']
['']
['']
['2·HR', 'GDP']
['']
['']
['']
['GDP']
['']
['']
['']
['HBP']
['']
['']
['']
['']
['']
['']
['']
['SB']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['3B']
['']
['']
['GDP']
['2B']
['SF']
['']
['']
['']
['']
['HR']
['']
['']
['']
['']
['HBP']
['']
['']
['2B']
['']
['']
['']
['']
['2B']
['']
['2B']
['']
['']
['']
['']
['2·2B']
['CS']
['HR']
['']
['']
['2B']
['IW']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['2B', 'SF']
['']
['HR']
['HR']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['SB']
['']
['']
['HR']
['CS']
['']
['']
['']
['SH']
['']
['']
['']
['SB']
['SF']
['']
['2B']
['2B']
['2B']
['']
['HR', 

['']
['']
['GDP']
['SH']
['']
['']
['']
['']
['']
['']
['']
['HBP']
['2B']
['2B', 'HBP']
['']
['HR', 'GDP']
['']
['SF']
['GDP']
['SF']
['']
['HR', '2B']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['HR']
['HR', 'GDP']
['']
['']
['HR', 'HBP', 'GDP']
['']
['2B', 'HBP']
['']
['']
['']
['']
['']
['']
['']
['SH']
['']
['IW']
['']
['']
['3B', 'GDP']
['']
['2B']
['SH']
['']
['']
['']
['']
['2B']
['2B', 'HBP']
['IW']
['SF']
['']
['HR']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['2B', 'SB']
['']
['GDP']
['']
['GDP']
['HR']
['2B', 'HBP']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['2B']
['IW']
['HR', 'GDP']
['']
['3B']
['']
['']
['HR']
['2B']
['']
['']
['']
['GDP']
['']
['GDP']
['HR']
['HR']
['HBP']
['GDP']
['']
['']
['']
['']
['']
['']
['']
['']
['CS']
['2B']
['HR']
['']
['']
['2B', 'HBP']
['']
['']
['GDP']
['']
['']
['']
['']
['']
['']
['']
['']
['HBP']
['']
['']
['']
['']
['']
['2B']
['2B']
['']
['']
['CS']
['']
['']
['']
['']
['2·2B']
['SB']
['']
['']
['GDP']
['']
[

In [50]:
HIT_df['HR'], HIT_df['2B'], HIT_df['3B'], HIT_df['HBP'], HIT_df['IBB'], HIT_df['SF'] = zip(*ultimate_hitOutcome_list)

/Users/tah96/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [51]:
HIT_df['Name'].unique()

array(['Victor Robles', 'Trea Turner', 'Juan Soto', 'Ryan Zimmerman',
       'Luis Garcia', 'Starlin Castro', 'Hernan Perez',
       'Andrew Stevenson', 'Jonathan Lucroy', 'Max Scherzer',
       'Kyle Finnegan', 'Wander Suero', 'Carter Kieboom', 'Kyle McGowin',
       'Yadiel Hernández', 'Daniel Hudson', 'Erick Fedde', 'Cody Wilson',
       'Sam Clay', 'Luis Avilan', 'Tres Barrera', 'Stephen Strasburg',
       'Tanner Rainey', 'Joe Ross', 'Yan Gomes', 'Jordy Mercer',
       'Patrick Corbin', 'Austin Voth', 'Brad Hand', 'Josh Bell',
       'Kyle Schwarber', 'Josh Harrison', 'Alex Avila', 'Paolo Espino',
       'Ryne Harper', 'Jon Lester', 'Will Harris', 'Jefry Rodriguez',
       'Justin Miller', 'Gerardo Parra', 'Kyle Lobstein',
       'Andres Machado', 'Humberto Arteaga', 'Alcides Escobar',
       'Jakson Reetz', 'Rene Rivera', 'Gabe Klobosits', 'Adrian Sanchez',
       'Josiah Gray', 'Mason Thompson', 'Riley Adams', 'Javy Guerra',
       'Sean Nolin', 'Lane Thomas', 'Patrick Murphy', 

In [52]:
HIT_df = HIT_df.replace("", '0')
HIT_df = HIT_df.loc[HIT_df['Name']!='0']
cleaned_positions = []
for position in HIT_df['Position']:
    pos_obj_to_str = str(position)
    whitespace_removed = pos_obj_to_str.strip()
    cleaned_positions.append(whitespace_removed)
HIT_df['GAME_ID'] = HIT_df['GAME_ID'].astype(str)
HIT_df['Position'] = cleaned_positions
HIT_df['AB'] = HIT_df['AB'].astype(int)
HIT_df['H'] = HIT_df['H'].astype(int)
HIT_df['R'] = HIT_df['R'].astype(int)
HIT_df['RBI'] = HIT_df['RBI'].astype(int)
HIT_df['BB'] = HIT_df['BB'].astype(int)
HIT_df['K'] = HIT_df['SO'].astype(int)
HIT_df['PA'] = HIT_df['PA'].astype(int)
HIT_df['Pit'] = HIT_df['Pit'].astype(int)
HIT_df['Str'] = HIT_df['Str'].astype(int)
HIT_df['A'] = HIT_df['A'].astype(int)
HIT_df['PO'] = HIT_df['PO'].astype(int)
HIT_df['HR'] = HIT_df['HR'].astype(int)
HIT_df['2B'] = HIT_df['2B'].astype(int)
HIT_df['3B'] = HIT_df['3B'].astype(int)
HIT_df['HBP'] = HIT_df['HBP'].astype(int)
HIT_df['IBB'] = HIT_df['IBB'].astype(int)
HIT_df['SF'] = HIT_df['SF'].astype(int)
bags_from_xbh = HIT_df['HR'] * 4 + HIT_df['2B'] * 2 + HIT_df['3B'] * 3
HIT_df['TB'] = bags_from_xbh + (HIT_df['H'] - HIT_df['HR'] - HIT_df['3B'] - HIT_df['2B'])
HIT_clean_df = HIT_df[['GAME_ID','Name','Position','AB','R','H','RBI','BB','K','PA','2B','3B','HR','TB','HBP','IBB','SF']]
HIT_player_game_list = HIT_clean_df['Name'].to_list()
#print(HIT_player_game_list)
HIT_playerID_list = []
for index, player in enumerate(HIT_player_game_list):
    try:
        #print(f'{player} is at {index}')
        new_player = player_csv.loc[player_csv['playerName'] == player,'playerID'].iloc[0]
        HIT_playerID_list.append(new_player)
    except IndexError:
        print(f'NO PLAYER LOCATED AT {index}')
HIT_clean_df['Name'] = HIT_playerID_list
HIT_ntf_df = HIT_clean_df.rename({'GAME_ID': 'gameId', 'Name': 'playerId', 'Position':'position','PA':'pPlateApp',
            'AB':'pAtBats','H':'pHits','R':'pRuns','K':'pStrikeouts','RBI': 'pRBI','TB':'pTB',
            '2B':'p2B','3B':'p3B','HR':'pHR','SF':'pSF','BB':'pBB','IBB':'pIBB','HBP':'pHBP'}, axis=1)

HIT_ntf_df

/Users/tah96/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,gameId,playerId,position,pAtBats,pRuns,pHits,pRBI,pBB,pStrikeouts,pPlateApp,p2B,p3B,pHR,pTB,pHBP,pIBB,pSF
0,1,roblvic01,CF,3,2,1,0,2,1,5,0,0,0,1,0,0,0
1,1,turntre01,SS,4,1,1,2,0,1,5,0,0,1,4,1,0,0
2,1,sotojua01,RF,5,0,1,1,0,1,5,0,0,0,1,0,0,0
3,1,zimmrya01,1B,4,0,2,0,0,1,4,0,0,0,2,0,0,0
4,1,garclui01,PR-2B,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2386,162,fedderi01,P,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2387,162,machand01,P,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2388,162,adamril01,PH,1,0,0,0,0,1,1,0,0,0,0,0,0,0
2389,162,finnkyl01,P,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
HIT_unique_gameId_list = HIT_ntf_df['gameId'].unique()

In [54]:
GAME_unique_gameId_list = GAME_df['gameId'].to_list()

In [55]:
for index, game in enumerate(HIT_unique_gameId_list):
    gameIDnewval = GAME_unique_gameId_list[index]
    HIT_ntf_df.loc[HIT_ntf_df['gameId'] == game, "gameId"] = gameIDnewval

In [56]:
HIT_ntf_df

,gameId,playerId,position,pAtBats,pRuns,pHits,pRBI,pBB,pStrikeouts,pPlateApp,p2B,p3B,pHR,pTB,pHBP,pIBB,pSF
0,20210406023001,roblvic01,CF,3,2,1,0,2,1,5,0,0,0,1,0,0,0
1,20210406023001,turntre01,SS,4,1,1,2,0,1,5,0,0,1,4,1,0,0
2,20210406023001,sotojua01,RF,5,0,1,1,0,1,5,0,0,0,1,0,0,0
3,20210406023001,zimmrya01,1B,4,0,2,0,0,1,4,0,0,0,2,0,0,0
4,20210406023001,garclui01,PR-2B,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2386,20211003043001,fedderi01,P,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2387,20211003043001,machand01,P,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2388,20211003043001,adamril01,PH,1,0,0,0,0,1,1,0,0,0,0,0,0,0
2389,20211003043001,finnkyl01,P,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [57]:
HIT_final_filtered = HIT_ntf_df.loc[(HIT_ntf_df['position']!='P') | (HIT_ntf_df['pPlateApp']!=0)]
HIT_final_filtered

,gameId,playerId,position,pAtBats,pRuns,pHits,pRBI,pBB,pStrikeouts,pPlateApp,p2B,p3B,pHR,pTB,pHBP,pIBB,pSF
0,20210406023001,roblvic01,CF,3,2,1,0,2,1,5,0,0,0,1,0,0,0
1,20210406023001,turntre01,SS,4,1,1,2,0,1,5,0,0,1,4,1,0,0
2,20210406023001,sotojua01,RF,5,0,1,1,0,1,5,0,0,0,1,0,0,0
3,20210406023001,zimmrya01,1B,4,0,2,0,0,1,4,0,0,0,2,0,0,0
4,20210406023001,garclui01,PR-2B,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2381,20211003043001,ruizkei01,C,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2382,20211003043001,kiebcar01,3B,3,0,1,0,1,0,4,1,0,0,2,0,1,0
2383,20211003043001,adonjoa01,P,2,0,0,0,0,2,2,0,0,0,0,0,0,0
2385,20211003043001,parrger01,PH,1,0,0,0,0,0,1,0,0,0,0,0,0,0


In [64]:
HIT_final_filtered.to_csv('hitting01032022.csv', index=False)  

In [ ]:
# PITCH_df = pd.DataFrame(PITCH_info, columns = PITCH_beg_columns)
# PITCH_df

In [ ]:
# PITCH_df.to_csv('pitchingTemp',index=False)

In [ ]:
# PITCH_player_list = PITCH_df['Pitching'].to_list()
# #print(HIT_player_game_list)
# PITCH_playerID_list = []
# for index, player in enumerate(PITCH_player_list):
#     try:
#         #print(f'{player} is at {index}')
#         new_player = player_csv.loc[player_csv['playerName'] == player,'playerID'].iloc[0]
#         PITCH_playerID_list.append(new_player)
#     except IndexError:
#         print(f'NO PLAYER LOCATED AT {index}')
# PITCH_df['Pitching'] = PITCH_playerID_list

# PITCH_df

In [ ]:
# PITCH_unique_gameId_list = PITCH_df['GAME_ID'].unique()
# GAME_unique_gameId_list = GAME_df['gameId'].to_list()

# for index, game in enumerate(PITCH_unique_gameId_list):
#     gameIDnewval = GAME_unique_gameId_list[index]
#     PITCH_df.loc[PITCH_df['GAME_ID'] == game, "GAME_ID"] = gameIDnewval

In [ ]:
# PITCH_df_final_pre = PITCH_df.rename({'GAME_ID': 'gameId', 'Pitching': 'playerId', 'SPFlag':'spFlag','IP':'pInnPit','H':'oppHits',
#             'R':'oppRuns','ER':'oppEarnRuns','Pit':'pitchesThrown','Str':'strikesThrown','HR':'oppHR','SO':'oppStrikeouts','BB': 'oppBB','IBB':'oppIBB',
#             'HBP':'oppHBP','GB':'oppGB','FB':'oppFB','LD':'oppLD','WPA':'wpa','cWPA':'cwpa','W':'winFlag','L':'lossFlag',
#             'H.1':'holdFlag','S':'saveFlag','BS':'blownSaveFlag'}, axis=1)

In [ ]:
# PITCH_df_final_pre.columns

In [ ]:
# PITCH_df_final_col = PITCH_df_final_pre[['gameId','playerId', 'spFlag','pInnPit', 'oppHits', 'oppRuns', 'oppEarnRuns', 'oppBB',
#        'oppStrikeouts', 'oppHR','pitchesThrown', 'strikesThrown','oppGB', 'oppFB', 'oppLD','wpa','cwpa','oppHBP', 'oppIBB', 'winFlag',
#        'lossFlag', 'holdFlag', 'saveFlag', 'blownSaveFlag']]

In [ ]:
# PITCH_df_final_col.columns

In [ ]:
# PITCH_df_final_col

In [ ]:
# PITCH_df_final_col.to_csv('pitching_122022', index=False)